In [28]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/workation-price-prediction-challengemachinehack/sample submission.csv
/kaggle/input/workation-price-prediction-challengemachinehack/Train.csv
/kaggle/input/workation-price-prediction-challengemachinehack/Test.csv


In [29]:
import numpy as np # linear algebra
from sklearn.feature_selection import SelectKBest,f_regression,mutual_info_regression
import pandas as pd 
from geopy.geocoders import Nominatim 
from geopy import distance 
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
import time
from sklearn.model_selection import *
from sklearn.metrics import *
from sklearn.linear_model import *
from wordcloud import WordCloud, STOPWORDS
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from nltk.tokenize import TreebankWordTokenizer
import matplotlib.pyplot as plt
from geopy.geocoders import Nominatim 
from geopy import distance 
geolocator = Nominatim(user_agent="geoapiExercises") 
%matplotlib inline
import re
from sklearn.metrics import mean_squared_log_error
def custom_metric(y_true, y_preds):
    return np.sqrt(mean_squared_log_error(y_true, y_preds))
def rmse(y_true, y_pred):
    return mean_squared_error(y_true, y_pred) ** 0.5

In [30]:
def distance_calc (x):
    try :
        p1 = geolocator.geocode(x[0]) 
        p2 = geolocator.geocode(x[1])

        # Get latitude and longitude 
        lat1, lon1 = (p1.latitude), (p1.longitude) 
        lat2, lon2 = (p2.latitude), (p2.longitude) 
        
        location1 = (lat1, lon1) 
        location2 = (lat2, lon2) 
        
        dist = distance.distance(location1,location2).km 
        
        return dist
    except:
        return np.nan
    
def rating(x):
    x =  x.lower()
    x.replace("one", '1') 
    x.replace("two", '1') 
    x.replace("three", '1') 
    x.replace("four", '4') 
    x.replace("five", '5') 
    lst = re.findall(r"[-+]?\d*\.\d+|\d+",x)
    lst = [float(i) for i in lst]
    if len(lst) == 0:
        return 3
    else:
        return sum(lst)/len(lst)
    
def tot_nit(x):
    try :
        
        ls = x.split('.')
        night = 0
        for i in ls :
            k1 = i.split(' ')

            k2 = int([x for x in k1 if re.compile(r'^.N$').match(x)][0][0])
            night = night + k2
        return night
    except :
        return 0

In [31]:

train = pd.read_csv('/kaggle/input/workation-price-prediction-challengemachinehack/Train.csv')
test = pd.read_csv('/kaggle/input/workation-price-prediction-challengemachinehack/Test.csv')
sample_sub = pd.read_csv('/kaggle/input/workation-price-prediction-challengemachinehack/sample submission.csv')

In [32]:
df = pd.concat([train, test], axis=0).reset_index(drop=True)
df.shape

(30000, 15)

In [33]:
df['d1'] = df['Destination'].apply(lambda x : x.split('|')[0])
df['d2'] = df['Destination'].apply(lambda x : x.split('|')[-1])

df['d1'] = np.where(df['d1']=='Tiruchirapally','Tiruchi',df['d1'])
df['d1'] = np.where(df['d1']=='Chikmangalur','Chikmagalur',df['d1'])
df['d2'] = np.where(df['d2']=='Tiruchirapally','Tiruchi',df['d2'])
df['d2'] = np.where(df['d2']=='Chikmangalur','Chikmagalur',df['d2'])


temp1 = df[['Start City','d1']].drop_duplicates(keep='first')
temp2 = df[['Start City','d2']].drop_duplicates(keep='first')

temp1['dist1'] = temp1.apply(lambda x : distance_calc (x) , axis=1)
temp2['dist2'] = temp2.apply(lambda x : distance_calc (x),axis=1)


temp1['country'] = temp1['d1'].apply(lambda x :geolocator.reverse(str(geolocator.geocode(x).latitude)+","+str(geolocator.geocode(x).longitude) ).raw['address'].get('country_code', '') )
temp1 = pd.get_dummies(temp1, columns = ['country'])

df1 = df[['Uniq Id','Start City','d1','d2']]

df1 = pd.merge(df1,temp1,how = 'left' , on=['Start City','d1'])
df1 = pd.merge(df1,temp2,how = 'left' , on=['Start City','d2'])


df1.head(3)

,Uniq Id,Start City,d1,d2,dist1,country_ae,country_bt,country_cn,country_cz,country_dk,...,country_lk,country_mu,country_my,country_np,country_nz,country_ru,country_sg,country_th,country_us,dist2
0,e788ab76d9d8cf1e6ed2f139645ca5d1,Mumbai,New Delhi,Chandigarh,1144.539256,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1349.851080
1,178f892630ce3e335a5a41d5d83937fd,New Delhi,Srinagar,Srinagar,646.667617,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,646.667617
2,f060f2954840503cc2fdaf495357b7df,New Delhi,Udaipur,Chittorgarh,568.255010,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,510.392507


In [34]:
df1 = df1.drop(['Start City','d1','d2'],axis=1)
df = pd.merge(df,df1,how = 'left' , on=['Uniq Id'])



In [35]:
df['Airline1'] = df['Airline'].apply(lambda x : x.split('|')[0])
df['rating'] = df['Hotel Details'].apply(lambda x : rating(x))
df['rating'] = np.where(df['rating'] > 5 , 3 ,df['rating'])
df['info'] = df[['Package Name','Destination','Itinerary','Places Covered','Hotel Details','Sightseeing Places Covered','Cancellation Rules','Airline']].agg('-'.join, axis=1)
df['ps'] = df['Package Type']+'-'+df['Start City']

# datetimindex variables :- https://pandas.pydata.org/pandas-docs/version/0.23/generated/pandas.DatetimeIndex.html

df['Date'] = pd.to_datetime(df['Travel Date'])
df['Year'] = pd.to_datetime(df['Date']).dt.year
df['Month'] = pd.to_datetime(df['Date']).dt.month

df['Week'] = pd.to_datetime(df['Date']).dt.week 
df['Quarter'] = pd.to_datetime(df['Date']).dt.quarter  

df['Is_quarter_end'] = pd.to_datetime(df['Date']).dt.is_quarter_end 
df['Day'] = pd.to_datetime(df['Date']).dt.day
df['Dayofweek'] = pd.to_datetime(df['Date']).dt.dayofweek
df['DayOfyear'] = pd.to_datetime(df['Date']).dt.dayofyear
df['Is_year_start'] = pd.to_datetime(df['Date']).dt.is_year_start 
df['Is_year_end'] = pd.to_datetime(df['Date']).dt.is_year_end
df['Semester'] = np.where(df['Quarter'].isin([1,2]),1,2)   
df['Is_weekday'] = np.where(df['Dayofweek'].isin([0,1,2,3,4]),1,0)
df['Is_month_start'] = pd.to_datetime(df['Date']).dt.is_month_start 
df['Is_month_end'] = pd.to_datetime(df['Date']).dt.is_month_end 
df['Is_quarter_start'] = pd.to_datetime(df['Date']).dt.is_quarter_start
df['Days_in_month'] = pd.to_datetime(df['Date']).dt.days_in_month 
del df['Travel Date']

df['tot_night'] = df['Itinerary'].apply(lambda x : tot_nit(x) )
df['places_covered_ct']  = df['Destination'].apply(lambda x: len(x.split('|')) )
df['Sightseeing Places Covered ct']  = df['Sightseeing Places Covered'].apply(lambda x: len(x.split('|')) )
df['Cancellation Rules ct'] = df['Cancellation Rules'].apply(lambda x : len(x.split()))

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:13: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  del sys.path[0]


In [36]:
df['package_per_airline'] = df.groupby(['Package Type'])['Airline1'].transform('nunique')

df['avg_night_per_package'] = df.groupby(['Package Type'])['tot_night'].transform('mean')
df['avg_night_per_ps'] = df.groupby(['ps'])['tot_night'].transform('mean')

df['avg_places_per_package'] = df.groupby(['Package Type'])['places_covered_ct'].transform('mean')
df['avg_places_per_ps'] = df.groupby(['ps'])['places_covered_ct'].transform('mean')

df['avg_sight_per_package'] = df.groupby(['Package Type'])['Sightseeing Places Covered ct'].transform('mean')
df['avg_sight_per_ps'] = df.groupby(['ps'])['Sightseeing Places Covered ct'].transform('mean')

df['avg_can_per_package'] = df.groupby(['Package Type'])['Cancellation Rules ct'].transform('mean')
df['avg_can_per_ps'] = df.groupby(['ps'])['Cancellation Rules ct'].transform('mean')


In [37]:
df = df.drop(['Package Name','d1','d2','Destination','Itinerary','Places Covered','Hotel Details','Sightseeing Places Covered','Cancellation Rules','Airline'],axis=1)
df.head(1)

,Uniq Id,Package Type,Start City,Flight Stops,Meals,Per Person Price,dist1,country_ae,country_bt,country_cn,...,Cancellation Rules ct,package_per_airline,avg_night_per_package,avg_night_per_ps,avg_places_per_package,avg_places_per_ps,avg_sight_per_package,avg_sight_per_ps,avg_can_per_package,avg_can_per_ps
0,e788ab76d9d8cf1e6ed2f139645ca5d1,Standard,Mumbai,2,3,11509.0,1144.539256,0,0,0,...,2,26,4.759148,4.919508,2.597234,2.691388,7.509118,7.68717,124.714845,116.724429


In [38]:
df['dist'] = np.abs(df['dist1']-df['dist2'])
df = pd.get_dummies(df, columns = ['Package Type', 'Start City','ps','Airline1'])
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30000 entries, 0 to 29999
Columns: 106 entries, Uniq Id to Airline1_Vistara
dtypes: bool(6), datetime64[ns](1), float64(13), int64(17), object(2), uint8(67)
memory usage: 9.9+ MB


In [39]:
count_vec = CountVectorizer(stop_words="english", analyzer='word', 
                            ngram_range=(1, 1), max_df=1.0, min_df=1, max_features=1100)
df_info = pd.DataFrame(count_vec.fit_transform(df['info']).todense())
df_info.columns = ['Model_Info_Top_' + str(c) for c in df_info.columns]
df = pd.concat([df, df_info], axis=1)

In [40]:
train, test  = df[:train.shape[0]].reset_index(drop=True), df[train.shape[0]:].reset_index(drop=True)

In [42]:

features = [c for c in train.columns if c not in ['Uniq Id' ,'Per Person Price', 'info','Date']]
target = train['Per Person Price']

In [43]:
selector = SelectKBest(f_regression, k=900).fit(train[features], target)
cols = selector.get_support(indices=True)
features1 = train[features].iloc[:,cols].columns

In [45]:

training_start_time = time.time()

folds = StratifiedKFold(n_splits = 10)
train_prediction = np.zeros(len(train))
test_prediction = np.zeros(len(test))


for fold_, (train_idx, val_idx) in enumerate(folds.split(train, pd.qcut(target, 10, labels=False))):
    
    print(f'\n---- Fold {fold_} -----\n')
    
    fold_start_time = time.time()
    
    X_train, y_train = train.iloc[train_idx][features1], target.iloc[train_idx]
    X_val, y_val = train.iloc[val_idx][features1], target.iloc[val_idx]
    X_test = test[features1]
    print(X_train.shape[1], X_val.shape[1])

    lgbm = LGBMRegressor(n_estimators=10000, num_leaves=70, max_depth=20
                        ,min_child_samples=10, learning_rate=0.01, colsample_bytree=0.8,subsample=0.5, reg_alpha=0, reg_lambda=20)  

    
    _ = lgbm.fit(X_train, np.log(y_train), eval_set = [(X_val, np.log(y_val))], verbose=100, early_stopping_rounds=200, eval_metric='rmse')

    train_prediction[val_idx] = np.exp(lgbm.predict(X_val))
    current_test_pred = np.exp(lgbm.predict(X_test))
    test_prediction += np.exp(lgbm.predict(X_test))/10
    
    
    print(f'\n Fold {rmse(np.log(y_val), np.log(train_prediction[val_idx]))}')
    
    fold_end_time = time.time()
    total_fold_time = int(fold_end_time - fold_start_time)
    
    print(f"\n->-> Fold ran for {(total_fold_time)//60} minutes {(total_fold_time)%60} seconds")
    

print(f'\ntrain score (validation fold) : {rmse(np.log(target), np.log(train_prediction))}')
training_end_time = time.time()
total_training_time = int(training_end_time - training_start_time)

print(f'\n->-> Total  time: {(total_training_time)//60} minutes ')

lgbm_prediction = test_prediction


---- Fold 0 -----

900 900
Training until validation scores don't improve for 200 rounds
[100]	valid_0's rmse: 0.313207	valid_0's l2: 0.0980989
[200]	valid_0's rmse: 0.241892	valid_0's l2: 0.0585117
[300]	valid_0's rmse: 0.216956	valid_0's l2: 0.0470697
[400]	valid_0's rmse: 0.204491	valid_0's l2: 0.0418166
[500]	valid_0's rmse: 0.196649	valid_0's l2: 0.0386709
[600]	valid_0's rmse: 0.191783	valid_0's l2: 0.0367808
[700]	valid_0's rmse: 0.188357	valid_0's l2: 0.0354783
[800]	valid_0's rmse: 0.185577	valid_0's l2: 0.0344388
[900]	valid_0's rmse: 0.183303	valid_0's l2: 0.0335998
[1000]	valid_0's rmse: 0.181245	valid_0's l2: 0.0328499
[1100]	valid_0's rmse: 0.179558	valid_0's l2: 0.032241
[1200]	valid_0's rmse: 0.17822	valid_0's l2: 0.0317622
[1300]	valid_0's rmse: 0.177174	valid_0's l2: 0.0313905
[1400]	valid_0's rmse: 0.176221	valid_0's l2: 0.0310539
[1500]	valid_0's rmse: 0.175299	valid_0's l2: 0.0307296
[1600]	valid_0's rmse: 0.174454	valid_0's l2: 0.0304342
[1700]	valid_0's rmse: 0.

In [46]:

training_start_time = time.time()

folds = StratifiedKFold(n_splits = 10)
train_prediction = np.zeros(len(train))
test_prediction = np.zeros(len(test))


for fold_, (train_idx, val_idx) in enumerate(folds.split(train, pd.qcut(target, 10, labels=False))):
    
    print(f'\n---- Fold {fold_} -----\n')
    
    fold_start_time = time.time()
    
    X_train, y_train = train.iloc[train_idx][features1], target.iloc[train_idx]
    X_val, y_val = train.iloc[val_idx][features1], target.iloc[val_idx]
    X_test = test[features1]
    print(X_train.shape[1], X_val.shape[1])

    xgb = XGBRegressor(n_estimators=1000, max_depth=12, learning_rate=0.05, colsample_bytree=0.45)

    
    _ = xgb.fit(X_train, np.log(y_train), eval_set = [(X_val, np.log(y_val))], verbose=100, early_stopping_rounds=200, eval_metric='rmse')

    train_prediction[val_idx] = np.exp(xgb.predict(X_val))
    current_test_pred = np.exp(xgb.predict(X_test))
    test_prediction += np.exp(xgb.predict(X_test))/10
    
    
    print(f'\n Fold {rmse(np.log(y_val), np.log(train_prediction[val_idx]))}')
    
    fold_end_time = time.time()
    total_fold_time = int(fold_end_time - fold_start_time)
    
    print(f"\n->-> Fold ran for {(total_fold_time)//60} minutes {(total_fold_time)%60} seconds")
    

print(f'\ntrain score (validation fold) : {rmse(np.log(target), np.log(train_prediction))}')
training_end_time = time.time()
total_training_time = int(training_end_time - training_start_time)

print(f'\n->-> Total  time: {(total_training_time)//60} minutes ')

xgb_prediction = test_prediction


---- Fold 0 -----

900 900
[0]	validation_0-rmse:8.81060
[100]	validation_0-rmse:0.19627
[200]	validation_0-rmse:0.17425
[300]	validation_0-rmse:0.17067
[400]	validation_0-rmse:0.16942
[500]	validation_0-rmse:0.16880
[600]	validation_0-rmse:0.16863
[700]	validation_0-rmse:0.16862
[778]	validation_0-rmse:0.16865

 Fold 0.16853677726261548

->-> Fold ran for 3 minutes 11 seconds

---- Fold 1 -----

900 900
[0]	validation_0-rmse:8.81889
[100]	validation_0-rmse:0.18686
[200]	validation_0-rmse:0.16454
[300]	validation_0-rmse:0.16057
[400]	validation_0-rmse:0.15881
[500]	validation_0-rmse:0.15783
[600]	validation_0-rmse:0.15724
[700]	validation_0-rmse:0.15716
[800]	validation_0-rmse:0.15715
[900]	validation_0-rmse:0.15721
[953]	validation_0-rmse:0.15733

 Fold 0.15707460135577

->-> Fold ran for 3 minutes 49 seconds

---- Fold 2 -----

900 900
[0]	validation_0-rmse:8.81494
[100]	validation_0-rmse:0.17573
[200]	validation_0-rmse:0.15518
[300]	validation_0-rmse:0.15201
[400]	validation_0-rmse

In [49]:

training_start_time = time.time()

folds = StratifiedKFold(n_splits = 10)
train_prediction = np.zeros(len(train))
test_prediction = np.zeros(len(test))


for fold_, (train_idx, val_idx) in enumerate(folds.split(train, pd.qcut(target, 10, labels=False))):
    
    print(f'\n---- Fold {fold_} -----\n')
    
    fold_start_time = time.time()
    
    X_train, y_train = train.iloc[train_idx][features1], target.iloc[train_idx]
    X_val, y_val = train.iloc[val_idx][features1], target.iloc[val_idx]
    X_test = test[features1]
    print(X_train.shape[1], X_val.shape[1])

    cat = CatBoostRegressor(n_estimators=2000, learning_rate=0.05, max_depth=9, rsm=0.5)

    
    _ = cat.fit(X_train, np.log(y_train), eval_set = [(X_val, np.log(y_val))], verbose=100, early_stopping_rounds=200)

    train_prediction[val_idx] = np.exp(cat.predict(X_val))
    current_test_pred = np.exp(cat.predict(X_test))
    test_prediction += np.exp(cat.predict(X_test))/10
    
    
    print(f'\n Fold {rmse(np.log(y_val), np.log(train_prediction[val_idx]))}')
    
    fold_end_time = time.time()
    total_fold_time = int(fold_end_time - fold_start_time)
    
    print(f"\n->-> Fold ran for {(total_fold_time)//60} minutes {(total_fold_time)%60} seconds")
    

print(f'\ntrain score (validation fold) : {rmse(np.log(target), np.log(train_prediction))}')
training_end_time = time.time()
total_training_time = int(training_end_time - training_start_time)

print(f'\n->-> Total  time: {(total_training_time)//60} minutes ')

cat_prediction = test_prediction


---- Fold 0 -----

900 900
0:	learn: 0.5253567	test: 0.5258151	best: 0.5258151 (0)	total: 106ms	remaining: 3m 32s
100:	learn: 0.2145369	test: 0.2265797	best: 0.2265797 (100)	total: 3.4s	remaining: 1m 4s
200:	learn: 0.1927714	test: 0.2086190	best: 0.2086190 (200)	total: 6.58s	remaining: 58.9s
300:	learn: 0.1776580	test: 0.1970527	best: 0.1970527 (300)	total: 9.64s	remaining: 54.4s
400:	learn: 0.1672044	test: 0.1898582	best: 0.1898582 (400)	total: 12.7s	remaining: 50.7s
500:	learn: 0.1591165	test: 0.1845939	best: 0.1845939 (500)	total: 15.8s	remaining: 47.4s
600:	learn: 0.1527201	test: 0.1809913	best: 0.1809913 (600)	total: 19s	remaining: 44.3s
700:	learn: 0.1471349	test: 0.1782502	best: 0.1782502 (700)	total: 22.2s	remaining: 41.1s
800:	learn: 0.1426138	test: 0.1762083	best: 0.1762083 (800)	total: 25.2s	remaining: 37.7s
900:	learn: 0.1385538	test: 0.1746820	best: 0.1746655 (897)	total: 28.6s	remaining: 34.9s
1000:	learn: 0.1348338	test: 0.1735669	best: 0.1735669 (1000)	total: 31.9s	rem

In [52]:
preds = lgbm_prediction*0.6 + xgb_prediction*0.3 + cat_prediction*0.1
sub = pd.DataFrame({'Per Person Price': preds})
sub.to_csv('cvec14_2.csv', index=False)

In [51]:
pd.DataFrame(preds)[0].describe()

count      9000.000000
mean      19645.620020
std       10418.672995
min        1835.631351
25%       12651.115034
50%       17581.734434
75%       24661.301808
max      122409.211346
Name: 0, dtype: float64